## `Model_3`

### Target:
- Data Augumentation

### Results:
- Parameters: 7.9 K
- Best Train Accuracy: 99.01
- Best Test Accuracy: 99.3

### Analysis:
- augumentation helps a lot


# Import Libraries

In [4]:
! nvidia-smi

zsh:1: command not found: nvidia-smi


In [5]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

## Data Transformations

We first start with defining our data transformations. We need to think what our data is and how can we augment it to correct represent images which it might not see otherwise.


In [6]:
# Train Phase transformations
train_transforms = transforms.Compose([
                                       transforms.RandomApply([transforms.CenterCrop(22), ], p=0.05),
                                       transforms.Resize((28, 28)),
                                       transforms.RandomRotation((-7.0, 7.0), fill=(1,)),
                                       transforms.ElasticTransform(10.0, 3.0),
                                       transforms.ToTensor(),
                                       transforms.Normalize((0.1307,), (0.3081,)) # The mean and std have to be sequences (e.g., tuples), therefore you should add a comma after the values.
                                       ])

# Test Phase transformations
test_transforms = transforms.Compose([
                                       transforms.ToTensor(),
                                       transforms.Normalize((0.1307,), (0.3081,))
                                       ])


# Dataset and Creating Train/Test Split

In [ ]:
from utils import prepare_mnist_data
train_loader, test_loader = prepare_mnist_data(
    train_transforms, test_transforms, batch_size=256)

# The model

In [23]:
!pip install torchsummary
from torchsummary import summary
from model import Net
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print(device)
model = Net(
    list('CcCCCcPCCCCGc'),
    [32, 16, 16, 16, 10, 10,
     16, 8, 8, 16,
     16, 10],
    dropout_value=0.03).to(device)
summary(model, input_size=(1, 28, 28))

cuda
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 26, 26]             288
              ReLU-2           [-1, 32, 26, 26]               0
       BatchNorm2d-3           [-1, 32, 26, 26]              64
           Dropout-4           [-1, 32, 26, 26]               0
            Conv2d-5           [-1, 16, 26, 26]             512
            Conv2d-6           [-1, 16, 24, 24]           2,304
              ReLU-7           [-1, 16, 24, 24]               0
       BatchNorm2d-8           [-1, 16, 24, 24]              32
           Dropout-9           [-1, 16, 24, 24]               0
           Conv2d-10           [-1, 10, 24, 24]             160
        MaxPool2d-11           [-1, 10, 12, 12]               0
           Conv2d-12           [-1, 16, 10, 10]           1,440
             ReLU-13           [-1, 16, 10, 10]               0
      BatchNorm2d-14           [-1

# Training and Testing

In [ ]:
from torch.optim.lr_scheduler import StepLR
from model import train, test
model =  Net().to(device)
optimizer = optim.Adam(model.parameters(), lr=0.01)
scheduler = StepLR(optimizer, step_size=4, gamma=0.05)

EPOCHS = 20
for epoch in range(EPOCHS):
    print("EPOCH:", epoch)
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)